In this scrupt we calculate the coorelation between median household income and 911 calls made per household for each neighborhood.

In [39]:
import pandas as pd
from pomegranate import *
import numpy as np

In [65]:
df = pd.read_csv("crimes_with_calls_2015_unique_demographics.csv", index_col=0)
bpd_df = pd.read_csv("./data/BPD_CRIME_DATA.csv", index_col=0)

In [62]:
df.shape

(9275, 35)

In [111]:
calls_df = pd.read_csv("./data/911_Calls_For_Service_2017.csv", index_col=0)
calls_df18 = pd.read_csv("./data/911_Calls_For_Service_2018.csv", index_col=0)
calls_df19 = pd.read_csv("./data/911_Calls_For_Service_2019.csv", index_col=0)

/Users/tuliprichard/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [117]:
calls_df = pd.concat([calls_df,calls_df18,calls_df19])

In [118]:
calls_df["Neighborhood"].nunique()

278

In [41]:
df.loc[df["Weapon"].isna(),"Weapon"] = "NONE"

In [43]:
df = df.drop(columns=["Unnamed: 0.1", "CrimeDate", "CrimeTime","CallId"])

In [44]:
df = df.dropna().reset_index(drop=True)

In [72]:
df['Neighborhood'] = df['Neighborhood'].replace('Patterson Park Neighborho', 'Patterson Park')
df['Inside/Outside'] = df['Inside/Outside'].replace('I', 'Inside')
df['Inside/Outside'] = df['Inside/Outside'].replace('O', 'Outside')

In [70]:
df["CallDescription"] = df['CallDescription'].str.upper()
df = df[df["CallDescription"]!= "`"]
df = df[df["CallDescription"]!= "911/HANGUP"]
df = df[df["CallDescription"]!= "911/NO VOICE"]

In [49]:
df['Priority'] = df['Priority'].str.upper()
df['Priority'] = df['Priority'].replace('EMERGENCY', 'HIGH')

In [87]:
nbhd_income = df[["Neighborhood","median_household_income",'num_households']].drop_duplicates().dropna()
nbhd_income

,Neighborhood,median_household_income,num_households
0,RAMBLEWOOD,47461.856928,104400.0
1,GLEN,39690.750670,102600.0
2,HIGHLANDTOWN,41538.153846,88000.0
4,CARROLLTON RIDGE,25151.903315,22500.0
5,JOHNSTON SQUARE,23625.756917,32450.0
...,...,...,...
3453,EASTWOOD,41538.153846,88000.0
3820,CEDARCROFT,78174.828597,347500.0
4417,DRUID HILL PARK,34879.263473,78950.0
5700,LOWER HERRING RUN PARK,38173.968254,45550.0


In [120]:
calls_df["Neighborhood"] = calls_df['Neighborhood'].str.upper()
calls_groupby_nbhd = calls_df.groupby(by="Neighborhood").count()[["recordId"]]
calls_groupby_nbhd = calls_groupby_nbhd.rename(columns={"recordId": "call_counts"})
# calls_groupby_nbhd["Neighborhood"] = calls_groupby_nbhd.index

In [124]:
result = pd.merge(nbhd_income, calls_groupby_nbhd, on = "Neighborhood", how = "inner")
result["calls_per_household"] = result.call_counts / result.num_households
result.sort_values(by="Neighborhood", ascending = True).head(5)[["Neighborhood", "median_household_income","calls_per_household"]]

,Neighborhood,median_household_income,calls_per_household
64,ABELL,32241.138221,0.083382
15,ALLENDALE,36701.906742,0.468165
187,ARCADIA,66010.877444,0.033961
87,ARLINGTON,33139.182464,0.737275
89,ARMISTEAD GARDENS,33179.511029,0.111464


In [122]:
result.calls_per_household.corr(result.median_household_income)

-0.36076924710314096

In [ ]:
result.